In [1]:
%pip install transformers datasets black evaluate[evaluator]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 9.2 MB/s 
     |████████████████████████████████| 362 kB 69.2 MB/s 
     |████████████████████████████████| 1.4 MB 51.3 MB/s 
     |████████████████████████████████| 53 kB 3.0 MB/s 
     |████████████████████████████████| 596 kB 64.8 MB/s 
     |████████████████████████████████| 101 kB 15.5 MB/s 
     |████████████████████████████████| 6.6 MB 53.1 MB/s 
     |████████████████████████████████| 212 kB 74.2 MB/s 
     |████████████████████████████████| 1.1 MB 51.6 MB/s 
     |████████████████████████████████| 140 kB 72.4 MB/s 
     |████████████████████████████████| 127 kB 75.1 MB/s 
     |████████████████████████████████| 96 kB 7.5 MB/s 
     |████████████████████████████████| 843 kB 59.8 MB/s 
     |████████████████████████████████| 271 kB 74.5 MB/s 
     |████████████████████████████████| 144 kB 73.7 MB/s 
     |██████████████████████████████

In [1]:
PERSIAN = True

In [2]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device


'cuda:0'

In [3]:
from transformers import AutoTokenizer

if PERSIAN:
    tokenizer = AutoTokenizer.from_pretrained(
        "HooshvareLab/bert-fa-base-uncased-clf-digimag"
    )
else:
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


Downloading:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [4]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [13]:
from datasets import load_dataset


In [16]:
import json

LABELS = [
    "مستقیم منفی",
    "غیر مستقیم منفی",
    "خنثی",
    "غیر مستقیم مثبت",
    "مستقیم مثبت",
]

LABEL_to_NUMBER = {
    LABELS[0]: 0,
    LABELS[1]: 1,
    LABELS[2]: 2,
    LABELS[3]: 3,
    LABELS[4]: 4,
}


def majority_vote(annotations):
    return list(
        map(
            lambda annotation_list: round(sum(annotation_list) / len(annotation_list)),
            annotations,
        )
    )


def numberize(annotation_list):
    return list(map(lambda x: LABEL_to_NUMBER[x], annotation_list))


def get_text_and_label(data):
    texts = []
    annotations = []
    for entry in data:
        texts.append(entry["text"])
        annotations.append(entry["annotations"])

    annotations = list(map(numberize, annotations))
    labels = majority_vote(annotations)
    return texts, labels


def get_dataset(data):
    texts, labels = get_text_and_label(data)
    # encodings = tokenizer(texts, truncation=True, padding=True)
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
    dataset = MyDataset(encodings, labels)
    return dataset


# def load_persian_data():
#     with open("dataset_annotated_finance.json") as f:
#         data = json.loads(f.read())
#         print(data.keys())
#         train = data["train"]
#         test = data["test"]
#         val = data["eval"]
#         print(len(train))
#         print(train[0].keys())
#         print(train[0])

#     train_dataset = get_dataset(train)
#     test_dataset = get_dataset(test)
#     val_dataset = get_dataset(val)
#     return train_dataset, test_dataset, val_dataset


def m(examples):
    annotations = examples["annotations"]
    annotations = list(map(numberize, annotations))
    labels = majority_vote(annotations)
    examples["label"] = labels
    return examples


def tokenize_max_length(examples):
    # print(examples["text"])
    # print(tokenizer(examples["text"], truncation=True, padding=True, max_length=512))
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=512)


def load_persian_data():
    train_dataset = load_dataset(
        "json", data_files="dataset_annotated_finance.json", field="train"
    )["train"]
    test_dataset = load_dataset(
        "json", data_files="dataset_annotated_finance.json", field="test"
    )["train"]
    val_dataset = load_dataset(
        "json", data_files="dataset_annotated_finance.json", field="eval"
    )["train"]
    print(train_dataset[0])
    train_dataset = train_dataset.map(m, batched=True)
    test_dataset = test_dataset.map(m, batched=True)
    val_dataset = val_dataset.map(m, batched=True)
    print(train_dataset[0])

    tokenized_train_data = train_dataset.map(tokenize_max_length, batched=True)
    tokenized_test_data = test_dataset.map(tokenize_max_length, batched=True)
    tokenized_val_data = val_dataset.map(tokenize_max_length, batched=True)
    return tokenized_train_data, tokenized_test_data, tokenized_val_data
    # return train_dataset, test_dataset, val_dataset


In [17]:
def tokenize(examples):
    return tokenizer(examples["text"], truncation=True)

def load_english_data():
    train_dataset = load_dataset("imdb", split='train')
    # train_data = load_dataset("imdb", split='train[:10%]')
    test_dataset = load_dataset("imdb", split='test')
    val_dataset = load_dataset("imdb", split='val')
    print(type(test_dataset))
    tokenized_train_data = train_dataset.map(tokenize, batched=True)
    tokenized_test_data = test_dataset.map(tokenize, batched=True)
    tokenized_val_data = val_dataset.map(tokenize, batched=True)
    return tokenized_train_data, tokenized_test_data, tokenized_val_data
    # return train_dataset, test_dataset, val_dataset

In [18]:
if PERSIAN:
    train_dataset, test_dataset, val_dataset = load_persian_data()
else:
    train_dataset, test_dataset, val_dataset = load_english_data()

print(type(train_dataset))
print(train_dataset[0])

Using custom data configuration default-2690770e2257ca68
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-2690770e2257ca68/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-3a9f07e7d1d5111a
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-3a9f07e7d1d5111a/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-d74a3ac25b4c6833
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-d74a3ac25b4c6833/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5)


...

{'text': 'آتش سوزی کارخانه جمیل نخ گسترده است / اعزام نیرو ادامه دارد\n____________\nمنصور شیشه فروش در گفت و گو با خبرنگار مهر با اشاره به حادثه آتش سوزی در کارخانه جمیل نخ اظهار داشت: این حادثه ساعت ۱۷ و ۳۰ دقیقه اتفاق افتاده و به نیروهای امدادی اعلام شده است. وی با بیان اینکه نیروهای امداد و نجات برای این حادثه اعزام شده است، تصریح کرد: دو دستگاه خودروی اطفا حریق از آتش نشانی شهرک صنعتی مورچه خورت و دو دستگاه خودرو به همراه نیروهای امدادی نیز از آتش نشانی شاهین شهر برای این حادثه اعزام شده است', 'annotations': ['غیر مستقیم منفی', 'خنثی', 'مستقیم منفی'], 'label': 1}


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

<class 'datasets.arrow_dataset.Dataset'>
{'text': 'آتش سوزی کارخانه جمیل نخ گسترده است / اعزام نیرو ادامه دارد\n____________\nمنصور شیشه فروش در گفت و گو با خبرنگار مهر با اشاره به حادثه آتش سوزی در کارخانه جمیل نخ اظهار داشت: این حادثه ساعت ۱۷ و ۳۰ دقیقه اتفاق افتاده و به نیروهای امدادی اعلام شده است. وی با بیان اینکه نیروهای امداد و نجات برای این حادثه اعزام شده است، تصریح کرد: دو دستگاه خودروی اطفا حریق از آتش نشانی شهرک صنعتی مورچه خورت و دو دستگاه خودرو به همراه نیروهای امدادی نیز از آتش نشانی شاهین شهر برای این حادثه اعزام شده است', 'annotations': ['غیر مستقیم منفی', 'خنثی', 'مستقیم منفی'], 'label': 1, 'input_ids': [2, 4662, 12596, 6307, 38526, 3234, 5223, 2806, 1013, 8207, 5077, 3251, 2924, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 9649, 8030, 3569, 2786, 3017, 1379, 3302, 2799, 6534, 4153, 2799, 3364, 2789, 5979, 4662, 12596, 2786, 6307, 38526, 3234, 3913, 2996, 1014, 2802, 5979, 3551, 4051, 1379, 3979, 4853, 3929, 6707, 1379, 2789, 4147, 18647, 3402, 2871, 2806, 1012, 2931, 2799, 35

In [19]:
# train_dataset[0]['text']

In [20]:
# from datasets.arrow_dataset import Dataset
# print(len(imdb))
# print(len(imdb.ReadInstruction('train', to=10, unit='%')))
# print(type(imdb['train']))
# print(imdb.keys())
# print(imdb['train'][0].keys())
# print(imdb['train'][0])

# print(type(tokenized_imdb))
# print(len(tokenized_imdb))
# print(tokenized_imdb.keys())
# print(tokenized_imdb['train'][0].keys())
# print(tokenized_imdb['train'][0])

In [21]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [22]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
        
if PERSIAN:
    pretrained_model = AutoModelForSequenceClassification.from_pretrained("HooshvareLab/bert-fa-base-uncased-clf-digimag", num_labels=5, ignore_mismatched_sizes=True).to(device)
else:
    pretrained_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2).to(device)

loading configuration file https://huggingface.co/HooshvareLab/bert-fa-base-uncased-clf-digimag/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/add0f9fcf409fd49365aed6e6dc5220f22f24b380b0b5b39fb65019353899bd1.3d2a0f8339fc4199b8545d3f992ee1b1f035693723de3c117dbe566e29022749
Model config BertConfig {
  "_name_or_path": "HooshvareLab/bert-fa-base-uncased-clf-digimag",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "finetuning_task": "digimag",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_positio

In [23]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=pretrained_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: annotations, text. If annotations, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1450
  Num Epochs = 5
  Instantaneous batch size per device = 8
  To

Step,Training Loss
500,0.601600


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=910, training_loss=0.42568216847849416, metrics={'train_runtime': 711.8776, 'train_samples_per_second': 10.184, 'train_steps_per_second': 1.278, 'total_flos': 1907606532864000.0, 'train_loss': 0.42568216847849416, 'epoch': 5.0})

In [26]:
model_name = 'my_model'

## Save Model

In [27]:
trainer.save_model(model_name)

Saving model checkpoint to my_model
Configuration saved in my_model/config.json
Model weights saved in my_model/pytorch_model.bin
tokenizer config file saved in my_model/tokenizer_config.json
Special tokens file saved in my_model/special_tokens_map.json


## Load Model

In [28]:
from transformers import AutoModel

# model2 = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5, ignore_mismatched_sizes=True)
# model2 = AutoModelForSequenceClassification.from_pretrained("my_model")
# model2 = PreTrainedModel.from_pretrained('my_model')

In [29]:
from transformers import pipeline

model = trainer.model

In [ ]:

# classifier = pipeline("sentiment-analysis")
# type(classifier)


In [30]:
# classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)
classifier = pipeline("text-classification", model=model.to("cpu"), tokenizer=tokenizer)
# classifier = pipeline(model=model, tokenizer=tokenizer)


In [31]:
# text = 'This is a very good movie. I admire the movie very much. It\'s casts were greate'
text = "That movie was a piece of shit"
text = "The film was horrible and I couldn'nt continue watching it"
# tokenized_text = tokenizer.tokenize(text)
# indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# tokens_tensor = torch.tensor([indexed_tokens])


In [35]:
# model(tokens_tensor)
# text = "ثبت قرار داد تجاری ایران و چین در جهت افزایش روابط اقتصادی برای اقتصاد ایران بسیار خوب است"
text = "آمریکا و انگلیس قرار داد جدیدی برای تحریم ایران ثبت کردند"
# text = "افزایش هزینه‌ی واردات مواد اولیه کاراخانه‌ها برای کشور خوب نیست"

classifier(text)


[{'label': 'LABEL_1', 'score': 0.7400696873664856}]

In [46]:
LABELMAPPING = {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
}
predictions = classifier(test_dataset["text"])
predictions = [LABELMAPPING[element["label"]] for element in predictions]


In [56]:
# from datasets import load_metric
from evaluate import evaluator
import evaluate

f1 = evaluate.load("f1")
# auc = evaluate.load("roc auc")
# metric = evaluate.load("f1", average='weighted')
# eval = evaluator("text-classification")


In [57]:
# results = eval.compute(model_or_pipeline=model, data=test_dataset, metric=metric, tokenizer=tokenizer,
#                        label_mapping=LABELMAPPING,
#                       #  label_mapping={"NEGATIVE": 0, "POSITIVE": 1},
#                        strategy="bootstrap", n_resamples=200)
# results = eval.compute(model_or_pipeline=model, data=test_dataset, metric=metric, tokenizer=tokenizer,
#                        label_mapping=LABELMAPPING)
# results = eval.compute(model_or_pipeline=model, data=test_dataset, metric=metric, tokenizer=tokenizer,
#                        label_mapping=LABELMAPPING, compute_parameters={"average":"weighted"})
references = test_dataset["label"]
print(predictions[:5])

result = f1.compute(predictions=predictions, references=references, average="weighted")


[3, 2, 2, 2, 3]


In [58]:
result

{'f1': 0.7297607655502392}